In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anish\\Desktop\\Kidney-Disease-Classification-MLflow\\research'

In [3]:

os.chdir("../")

In [4]:

%pwd

'c:\\Users\\anish\\Desktop\\Kidney-Disease-Classification-MLflow'

In [5]:
# import dagshub
# dagshub.init(repo_owner='AnishaChoubey', repo_name='Kidney-Disease-Classification-MLflow', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/AnishaChoubey/Kidney-Disease-Classification-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="AnishaChoubey"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2254846308235d831836322ec845966deac15585"



In [6]:
import tensorflow as tf


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-dataset",
            mlflow_uri="https://dagshub.com/AnishaChoubey/Kidney-Disease-Classification-MLflow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model= tf.keras.models.load_model(path)

        
        model.compile(
            optimizer="adam",
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )
        return model
    
     
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        # ✅ Save model locally first to avoid filepath issues
        local_model_path = Path("artifacts/training/model_for_mlflow.h5")
        self.model.save(local_model_path)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            # Log the model artifact
            # mlflow.log_artifact(str(local_model_path), artifact_path="model")

            # Optional: register model if not a file-based tracking URI
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    artifact_path="model",
                    registered_model_name="VGG16Model",
                    keras_model_kwargs={"save_format": "h5"}
                )

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e



DEBUG YAML RAW TYPE: <class 'dict'>
DEBUG YAML RAW CONTENT: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1SObwCsPjXeuAeWnyTMcw9Z_LWBhTGCkc/view?usp=drive_link', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
DEBUG YAML RAW TYPE: <class 'dict'>
DEBUG YAML RAW CONTENT: {'AUGMENTATION': False, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.08}


Found 1002 images belonging to 2 classes.


c:\Users\anish\Desktop\Kidney-Disease-Classification-MLflow\Kidney\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 314s 5s/step - accuracy: 0.7984 - loss: 0.4407


2025/09/16 12:04:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/09/16 12:05:26 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\anish\AppData\Local\Temp\tmpzukp1znv\model, flavor: tensorflow), fall back to return ['tensorflow==2.17.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\anish\Desktop\Kidney-Disease-Classification-MLflow\Kidney\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/09/16 12:20:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation